# Introduction to Ensemble Learning
## Definition of Ensemble Learning and the Problem of Overfitting
Ensemble learning is a powerful machine learning paradigm where multiple models are trained to solve the same problem and combined to obtain better results. It is particularly important for improving the accuracy and robustness of models and overcoming issues like overfitting, where a model learns the training data too well but fails to generalize to unseen data.



## Real-world Relevance
Ensemble learning is widely used in real-world applications for its robustness and accuracy. For example, it is used in finance for risk prediction, in healthcare for disease detection, and in e-commerce for recommendation systems. The main advantage is its ability to harness the power of multiple models to provide better predictions than individual models.



## Types of Ensembles
There are several types of ensemble methods, including:
1. **Bagging**: Builds multiple models independently and combines them by averaging predictions (e.g., Random Forests).
2. **Boosting**: Builds models sequentially, with each model trying to correct the mistakes of its predecessor (e.g., AdaBoost, Gradient Boosting).
3. **Stacking**: Combines multiple models using a meta-model to improve predictions.
Each type of ensemble has its own method of model integration, with varying degrees of complexity and computational cost.


# 2. Bagging and Random Forests


Bagging, short for Bootstrap Aggregating, is an ensemble technique that aims to improve the accuracy and stability of machine learning algorithms. It involves creating multiple subsets of a dataset with replacement (bootstrap) and training a model on each subset. The final prediction is obtained by averaging the predictions (for regression) or voting (for classification). Bagging helps reduce variance, thereby mitigating the risk of overfitting.



## Synthetic Data (don't need to run this)

I used `drawdata` to generate the data for the *Explore Bagging* section. It is a useful library to create simple datasets.

In [ ]:
import drawdata

widget = drawdata.ScatterWidget()
widget


In [61]:
data = widget.data_as_pandas

In [ ]:
data

In [66]:

data.to_csv('data/simple.csv', index=False)

In [ ]:
data.plot.scatter(x='x', y='y')

## Explore Bagging

In [ ]:
# make 6 different random forests with a single tree using different seed
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# hide warnings
import warnings
warnings.filterwarnings('ignore')

layouts = '''
123
456
'''
fig, axs = plt.subplot_mosaic(layouts, figsize=(8, 4))

n_trees = [1]*6
data = pd.read_csv('data/simple.csv')
for i, n_tree in enumerate(n_trees, start=1):
    ax = axs[str(i)]
    data.plot.scatter(x='x', y='y', ax=ax, title=f'n_trees={n_tree} seed={i}', s=1)
    # change random_state to get different trees
    clf = RandomForestRegressor(n_estimators=n_tree, max_depth=3, max_samples=.01, random_state=i)
    clf.fit(data[['x']], data['y'])
    x = pd.DataFrame({'x': np.linspace(data.x.min(), data.x.max(), 100)})
    y = clf.predict(x)
    ax.plot(x, y, c='red')

fig.tight_layout()
    


In [ ]:
# illustrate random forest vs single tree
# note that making max_samples very small is a way to make the trees more different
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# hide warnings
import warnings
warnings.filterwarnings('ignore')

layouts = '''
123
456
'''
fig, axs = plt.subplot_mosaic(layouts, figsize=(8, 4))

n_trees = [1, 2, 5, 10, 20 , 50]
data = pd.read_csv('data/simple.csv')
for i, n_tree in enumerate(n_trees, start=1):
    ax = axs[str(i)]
    data.plot.scatter(x='x', y='y', ax=ax, title=f'n_trees={n_tree}', s=1)
    # play with max_depth and max_samples to see how they affect the trees
    rf = RandomForestRegressor(n_estimators=n_tree, max_depth=3, max_samples=.01, random_state=42)
    rf.fit(data[['x']], data['y'])
    x = pd.DataFrame({'x': np.linspace(data.x.min(), data.x.max(), 100)})

    # loop over trees and plot them
    for i, tree in enumerate(rf.estimators_):
        y = tree.predict(x)
        ax.plot(x, y, c='grey', alpha=.9)
    y = rf.predict(x)
    ax.plot(x, y, c='red')

fig.tight_layout()


In [ ]:
from ipywidgets import interact, IntSlider, FloatSlider
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# hide warnings
import warnings
warnings.filterwarnings('ignore')


def plot_random_forest(data_path, max_depth, max_samples):
  """
  Plots random forest predictions for different numbers of trees.

  Args:
      data_path (str): Path to the CSV file containing data.
      max_depth (int): Maximum depth of the trees.
      max_samples (float): Proportion of samples to draw for each tree.
  """
  layouts = '''
  123456
  '''
  plt.close('all')
  fig, axs = plt.subplot_mosaic(layouts, figsize=(18, 4))
  n_trees = [1, 2, 5, 10, 20, 50]
  data = pd.read_csv(data_path)
  for i, n_tree in enumerate(n_trees, start=1):
      ax = axs[str(i)]
      data.plot.scatter(x='x', y='y', ax=ax, title=f'n_trees={n_tree}', s=1)

      rf = RandomForestRegressor(n_estimators=n_tree, 
                                 max_depth=max_depth, 
                                 max_samples=max_samples, 
                                 random_state=42)
      rf.fit(data[['x']], data['y'])
      x = pd.DataFrame({'x': np.linspace(data.x.min(), data.x.max(), 100)})

      # loop over trees and plot them
      for i, tree in enumerate(rf.estimators_):
          y = tree.predict(x)
          ax.plot(x, y, c='grey', alpha=.9)
      y = rf.predict(x)
      ax.plot(x, y, c='red')
  plt.show()


# Define sliders
max_depth_slider = IntSlider(min=1, max=10, value=3, description="Max Depth")
max_samples_slider = FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description="Max Samples")

# Update the plot whenever the slider values change
interact(plot_random_forest, 
         data_path="data/simple.csv", 
         max_depth=max_depth_slider, 
         max_samples=max_samples_slider)

# as we raise the depth, the trees become more complex
# as we raise the samples, the prediction becomes smoother

## Random Forest Example

A Random Forest is a popular ensemble method that utilizes bagging. It builds a multitude of decision trees at training time and outputs the mode of the classes (classification) or mean prediction (regression) of the individual trees. It is known for its robustness and ease of use without extensive hyperparameter tuning.


In [3]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import set_config

set_config(transform_output='pandas')

In [ ]:
# Load the dataset
adult_data = fetch_openml(data_id=1590)  # Adult census income dataset
X = adult_data.data
y = adult_data.target

In [16]:
# create local parquet file
(X
 .assign(y=y)
 .to_parquet('data/adult.parquet')
)

In [4]:
adult = pd.read_parquet('data/adult.parquet')
X = (adult.drop(columns='y'))
y = adult.y
     

In [ ]:
X

In [ ]:
y

In [ ]:
# check for missing values
X.isna().sum()

In [ ]:
X.describe(include='all').T

In [10]:
# check out my Applied Machine Learning: Feature Engineering course for more details
# make pipeline to handle categorical data
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

categorical_features = X.select_dtypes(include=['category']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

In [ ]:
# using a technique called one-hot encoding (or dummy encoding) to encode the categorical variables
# Note that age is numerical so we don't want to apply one-hot encoding to it
categorical_transformer.fit_transform(X.iloc[:, :2])

In [ ]:
# only encode the categorical features
categorical_transformer.fit_transform(X[categorical_features])

In [ ]:
# try just a decision tree
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# pipeline with decision tree
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Make predictions
predictions = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f'Model Accuracy: {accuracy:.3f}')



In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# pipeline with random forest
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Make predictions
predictions = pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f'Model Accuracy: {accuracy:.3f}')

## Parameter Tuning for Random Forest

While Random Forest is quite robust, tuning its hyperparameters can significantly boost performance. Key parameters include the number of trees (n_estimators), the maximum depth of each tree (max_depth), and the number of features to consider for each split (max_features).

In [ ]:
# tune number of trees
import pandas as pd

scores = []
training_scores = []
n_estimators = [10, 50, 100, 150, 200, 250, 300]
for n in n_estimators:
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(n_estimators=n, random_state=42))
    ])
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    scores.append(accuracy) 
    training_predictions = pipeline.predict(X_train)
    training_accuracy = accuracy_score(y_train, training_predictions)
    training_scores.append(training_accuracy)

(pd.DataFrame({'n_estimators': n_estimators, 'accuracy': scores, 'training_accuracy': training_scores})
 .set_index('n_estimators')
    .plot(title='Random Forest Classifier Accuracy vs Number of Trees', ylabel='Accuracy'))




## Challenge: Tune Random Forest Parameters

Task: Improve the model accuracy by tuning the 'max_depth' parameter.


## Solution: Tune Random Forest Parameters


# 3. Boosting and Gradient Boosting



## The Concept of Boosting

The key idea in boosting is to train models sequentially. With each model, the algorithm attempts to fix the errors made by the previous models. The base models are often simple, like decision stumps, and combined to form a stronger learner. Boosting is particularly effective in reducing bias and variance in the model predictions.

## AdaBoost: A Simple Boosting Example

AdaBoost (Adaptive Boosting) is a popular boosting algorithm. It is a meta-learner and can use different base learners.
We will use a Decision Tree Classifier as the base learner for AdaBoost in this example.

- First, it trains a model
- For subsequent models, it focuses more on the misclassified samples from the previous model (gives them a larger weight)

The final prediction is a weighted sum of the predictions from all the models.


In [ ]:
# illustrate random forest vs single tree
# note that making max_samples very small is a way to 
# make the trees more different
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression

# hide warnings
import warnings
warnings.filterwarnings('ignore')

layouts = '''
123
456
'''
fig, axs = plt.subplot_mosaic(layouts, figsize=(10, 4))

n_trees = [1, 2, 5, 10, 20 , 50]
data = pd.read_csv('data/simple.csv')
for i, n_tree in enumerate(n_trees, start=1):
    ax = axs[str(i)]
    data.plot.scatter(x='x', y='y', ax=ax, title=f'n_trees={n_tree}', s=1)
    # play with max_depth and max_samples to see how they affect the trees
    clf = AdaBoostRegressor(n_estimators=n_tree, estimator=DecisionTreeRegressor(max_depth=3), random_state=42)
    #clf = AdaBoostRegressor(n_estimators=n_tree, random_state=42, estimator=LinearRegression())
    clf.fit(data[['x']], data['y'])
    x = pd.DataFrame({'x': np.linspace(data.x.min(), data.x.max(), 100)})

    # plot just the last tree in grey
    y = clf.estimators_[-1].predict(x)
    ax.plot(x, y, c='grey', alpha=.9)
    # plot the final prediction in red
    y = clf.predict(x)
    ax.plot(x, y, c='red')

fig.tight_layout()  

# note that in tree 2 the grey line is fixing the left side.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', AdaBoostClassifier())
    ])

# Fit 
pipeline.fit(X_train, y_train)

# Evaluate
accuracy = accuracy_score(y_test, pipeline.predict(X_test))
print(f'Model Accuracy: {accuracy:.3f}')


## Gradient Boosting

Gradient Boosting is another boosting algorithm that uses a gradient descent approach to minimize the loss of the combined model. 
Rather than focusing on the misclassified samples, Gradient Boosting fits the new model to the residual errors of the previous model.
Each new model aims to correct the mistakes of the previous ones by moving in the direction of the gradient that reduces loss.

In Gradient Boosting, decision trees are trained sequentially to fit the gradient of the loss function, perfecting errors from previous models. It's powerful for both regression and classification tasks.

In [ ]:
# illustrate random forest vs single tree
# note that making max_samples very small is a way to make the trees more different
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

# hide warnings
import warnings
warnings.filterwarnings('ignore')

layouts = '''
123
456
'''
fig, axs = plt.subplot_mosaic(layouts, figsize=(10, 4))

n_trees = [1, 2, 5, 10, 20 , 50]
data = pd.read_csv('data/simple.csv')
for i, n_tree in enumerate(n_trees, start=1):
    ax = axs[str(i)]
    data.plot.scatter(x='x', y='y', ax=ax, title=f'n_trees={n_tree}', s=1)
    # play with max_depth and max_samples to see how they affect the trees
    clf = GradientBoostingRegressor(n_estimators=n_tree, random_state=42, max_depth=3)
    clf.fit(data[['x']], data['y'])
    x = pd.DataFrame({'x': np.linspace(data.x.min(), data.x.max(), 100)})
    # plot just the last tree in grey
    # GradientBoostingRegressor stores the trees in a numpy array
    y = clf.estimators_.flatten()[-1].predict(x)
    ax.plot(x, y, c='grey', alpha=.9)
    # plot the final prediction in red
    y = clf.predict(x)
    ax.plot(x, y, c='red')

fig.tight_layout()  

# the scikit learn implementation uses a dummy regressor initially
# each tree corrects the previous (thats why the grey line is bounding 0)
# You want to use "weak learners" so that you don't overfit (try changing the max_depth to 10)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', GradientBoostingClassifier())
    ])

# Fit 
pipeline.fit(X_train, y_train)

# Evaluate
print(f'Model Accuracy: {accuracy:.3f}')


## Hyperparameter Tuning for Boosting Models
Tuning hyperparameters is critical for maximizing the performance of boosting models.

Hyperparameters for AdaBoost include:

- `n_estimators`: Number of boosting stages to be run. (1, inf)
- `learning_rate`: Contribution of each model to the final prediction. (number between 0 and 1)
- `estimator`: The base estimator to fit on the training data. (Can set additional hyperparameters on the base estimator)

Key hyperparameters for Gradient Boosting include:

- `learning_rate`: Contribution of each tree to the final prediction. (0, 1]
- `n_estimators`: Number of boosting stages to be run. (1, inf)
- `max_depth`: Maximum depth of the individual trees. (1, inf)
- `subsample`: Fraction of samples used to fit the individual base learners. (0, 1]



In [ ]:
# tune number of trees
import pandas as pd

scores = []
training_scores = []
n_estimators = [1, 5, 10, 20, 50, 70, 100, 150, 200]
for n in n_estimators:
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', GradientBoostingClassifier(n_estimators=n, random_state=42))
    ])
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    scores.append(accuracy) 
    training_predictions = pipeline.predict(X_train)
    training_accuracy = accuracy_score(y_train, training_predictions)
    training_scores.append(training_accuracy)

(pd.DataFrame({'n_estimators': n_estimators, 'accuracy': scores, 'training_accuracy': training_scores})
 .set_index('n_estimators')
    .plot(title='Gradient Boosting Classifier Accuracy vs Number of Trees', ylabel='Accuracy'))




## Challenge: Tune AdaBoost Model

Your task is to optimize the `max_depth` parameter of an AdaBoost model to achieve higher accuracy.


## Solution: Tune AdaBoost Model

#  XGBoost

XGBoost, short for eXtreme Gradient Boosting, is an advanced implementation of gradient boosting developed by Tianqi Chen. It is known for its efficiency, speed, and predictive accuracy, making it a popular choice among data scientists.


Why use XGBoost vs the Gradient Boosting Classifier in scikit-learn?
- XGBoost is specifically designed to optimize computational speed and model performance.
- Has regularization techniques to avoid overfitting.
- Supports parallel processing and distributed computing for large datasets.
- Provides a variety of objective functions and evaluation criteria for regression, classification, and ranking problems.
- Handles missing data.
- Handles categorical features.




## Hands-on Coding with XGBoost

To start using XGBoost, ensure that the XGBoost library is installed in your Python environment. 

In [ ]:
# illustrate random forest vs single tree
# note that making max_samples very small is a way to make the trees more different
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from xgboost import XGBRegressor


# hide warnings
import warnings
warnings.filterwarnings('ignore')

layouts = '''
123
456
'''
fig, axs = plt.subplot_mosaic(layouts, figsize=(8, 4))

n_trees = [1, 2, 5, 10, 20 , 50]
data = pd.read_csv('data/simple.csv')
for i, n_tree in enumerate(n_trees, start=1):
    ax = axs[str(i)]
    data.plot.scatter(x='x', y='y', ax=ax, title=f'n_trees={n_tree}', s=1)
    # play with max_depth and max_samples to see how they affect the trees
    rf = XGBRegressor(n_estimators=n_tree, #estimator=DecisionTreeRegressor(max_depth=3),
                      max_depth=2,
                       random_state=42)
    rf.fit(data[['x']], data['y'])
    x = pd.DataFrame({'x': np.linspace(data.x.min(), data.x.max(), 100)})

    # loop over trees and plot them
    for z, j in enumerate(range(n_tree)):
        y = rf.predict(x, iteration_range=(j, j+1))
        # increase alpha based on z to make latter trees more visible
        ax.plot(x, y, c='grey', alpha=(z+1)/n_tree)
    y = rf.predict(x)
    # plot the final prediction in red
    ax.plot(x, y, c='red')

plt.tight_layout()

# you can see n tree = 2 the right side is fixed by the second tree

In [ ]:
# no pipeline needed! it handles categorical data (and missing values) internally
from xgboost import XGBClassifier

model = XGBClassifier(enable_categorical=True)
model.fit(X_train, y_train=='>50K')
accuracy = accuracy_score(y_test=='>50K', model.predict(X_test))
print(f'Model Accuracy: {accuracy:.3f}')


## Hyperparameter Tuning for XGBoost
XGBoost is rich in hyperparameters that, when tuned properly, can significantly enhance model performance. The primary tuning parameters include:
- `max_depth`: Maximum depth of a tree. Increasing this value makes the model more complex and, hence, more likely to overfit. [0, inf)
- `eta`: Step size shrinkage is used in the update to prevent overfitting. (0, 1]
- `subsample`: Proportion of training instances used in trees; reduces overfitting. (0, 1]
- `colsample_bytree`: The fraction of features used from each column helps with overfitting. (0, 1]
- `gamma`: Minimum loss reduction required to make a further partition on a leaf node of the tree. L0 regularization term. [0, inf)



In [ ]:
scores = []
training_scores = []
values = [1, 2, 3, 6, 10, 15]
for value in values:
    model = XGBClassifier(enable_categorical=True, max_depth=value)
    model.fit(X_train, y_train=='>50K')
    scores.append(accuracy_score(y_test=='>50K', model.predict(X_test)))
    training_scores.append(accuracy_score(y_train=='>50K', model.predict(X_train)))

pd.DataFrame({'max_depth': values, 'accuracy': scores, 'training_accuracy': training_scores}).set_index('max_depth').plot(title='XGBoost Classifier Accuracy vs Depth of Trees', ylabel='Accuracy')

In [ ]:
xg_tuned = XGBClassifier(enable_categorical=True, max_depth=3)
xg_tuned.fit(X_train, y_train=='>50K')
accuracy = accuracy_score(y_test=='>50K', xg_tuned.predict(X_test))
print(f'Model Accuracy: {accuracy:.3f}')

## Challenge: Tune XGBoost Model

What value of `gamma` maximizes accuracy?



## Solution: Tune XGBoost Model



# Stacking
## Concept of Stacking
Stacking, also known as stacked generalization, is an ensemble machine learning algorithm. Unlike other ensemble methods, which are typically based on a single type of model, stacking focuses on combining different base models to improve predictive performance. The core idea is to train a meta-model (also known as a blender or second-level model) to combine predictions from multiple base models.



## Hands-on Coding with Stacking

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier

# Define base models
estimators = [
    ('dt', DecisionTreeClassifier(max_depth=3)),
    ('nn', KNeighborsClassifier(n_neighbors=3)),
]
model = StackingClassifier(estimators=estimators)

pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor)
    ])

model.fit(pipeline.fit_transform(X_train), y_train)
accuracy = accuracy_score(y_test, model.predict(pipeline.transform(X_test)))
print(f'Model Accuracy: {accuracy:.3f}')

## Evaluation: Stacking vs Individual Models

Now, let's compare the performance of the stacking model with its individual base models. Observe how the stacked model often outperforms the single models by combining their strengths.


In [ ]:

# Evaluate each individual model
for name, clf in estimators:
    clf.fit(pipeline.fit_transform(X_train), y_train)
    test_score = clf.score(pipeline.transform(X_test), y_test)
    print(f"{name} model score: {test_score:.4f}")

# Evaluate stacking model
test_score = model.score(pipeline.transform(X_test), y_test)
print(f"Stacking model score: {test_score:.4f}")


## Challenge: Create a Stacked Model
Try creating your own stacked model using different base models. You can experiment with models like Support Vector Machines, Decision Trees with different depths, or even different ensemble methods.



## Solution: Create a Stacked Model

